In [3]:
import cmethods
import numpy as np

cmethods.reconstruct(cmethods.MACHETE, np.array([1, 2, 3, 1, 5, 6, 7, 8, 9, 101.1], dtype="float"))

array([  1.   ,   2.998,   2.998,   1.   ,   4.996,   6.994,   6.994,
         8.992,   8.992, 100.9  ])